# Swedish Property Market Analysis Notebook

This notebook contains an extensive analysis of the Swedish property market that allows for better purchase and sales decisions.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
import ipywidgets as widgets

In [3]:
con = sqlite3.connect('data/hemnet.db')
closed_listings = pd.read_sql_query("select * from closed_listings", con)
closed_listings

,listing_url,address,city,listing_type,closing_price,closing_date,price_per_sqm,asking_price,price_development,form_of_assignment,...,floor,year_of_construction,monthly_fee,operating_costs,land_area_in_sqm,land_lease_fee,auxiliary_area_in_sqm,patio,raw_text,agency
0,https://www.hemnet.se/salda/villa-7rum-ulvkall...,Sörmovägen 13,"Ulvkälla, Härjedalens kommun",Villa,1675000,2022-07-28,12316,1350000,325000,Äganderätt,...,Unknown,1975,-1,-1,959,-1,48,0,Prisinformation\nPris per kvadratmeter\n12 316...,Kontakta mäklarkontoret\nHemverket AB
1,https://www.hemnet.se/salda/fritidsboende-4rum...,Flo klev Svensgården 12,"Grästorp, Grästorps kommun",Fritidshus,685000,2022-07-28,11230,650000,35000,Äganderätt,...,Unknown,1890,-1,-1,2833,-1,-1,0,Prisinformation\nPris per kvadratmeter\n11 230...,Kontakta mäklarkontoret\nHemverket AB
2,https://www.hemnet.se/salda/lagenhet-2rum-cent...,"Drabantgatan 52, bv","Centralt, Upplands Väsby kommun",Lägenhet,2000000,2022-07-28,43478,1995000,0,Bostadsrätt,...,"1 av 4, hiss finns",2019,2382,-1,-1,-1,-1,1,Prisinformation\nPris per kvadratmeter\n43 478...,Kontakta mäklaren\nKarin Sökare\nHusmanHagberg...
3,https://www.hemnet.se/salda/lagenhet-1rum-lund...,Ulfsparregatan 9E,"Lunden, Göteborgs kommun",Lägenhet,2380000,2022-07-28,64324,2150000,230000,Bostadsrätt,...,"3 av 4, hiss finns ej",1943,2666,-1,-1,-1,-1,0,Prisinformation\nPris per kvadratmeter\n64 324...,Kontakta mäklaren\nNatasha Vasic\nErik Olsson ...
4,https://www.hemnet.se/salda/villa-7rum-snostor...,Björkås Björkbacka 341,"Snöstorp, Halmstads kommun",Villa,5800000,2022-07-28,17313,5550000,250000,Äganderätt,...,Unknown,1900,-1,-1,1312,-1,-1,0,Prisinformation\nPris per kvadratmeter\n17 313...,Kontakta mäklaren\nPatrik Frimodig\nMOHV Halmstad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2267,https://www.hemnet.se/salda/villa-5rum-frano-k...,Envägen 8,"Frånö, Kramfors kommun",Villa,1550000,2022-07-20,14762,1350000,200000,Äganderätt,...,Unknown,1980,-1,-1,1017,-1,105,0,Prisinformation\nPris per kvadratmeter\n14 762...,Kontakta mäklaren\nMatilda Sjölund\nSkandiaMäk...
2268,https://www.hemnet.se/salda/lagenhet-4rum-husi...,Västra stationsgatan 12,"Husie, Malmö kommun",Lägenhet,2300000,2022-07-20,22330,2595000,-295000,Bostadsrätt,...,"1 av 2, hiss finns ej",1986,6616,-1,-1,-1,-1,1,Prisinformation\nPris per kvadratmeter\n22 330...,Kontakta mäklaren\nYosef Halim\nBjurfors Lund ...
2269,https://www.hemnet.se/salda/lagenhet-2rum-kors...,Tegnérgatan 3,"Korsängsgärdet, Västerås kommun",Lägenhet,1700000,2022-07-20,22667,1695000,0,Bostadsrätt,...,"2, hiss finns",-1,3524,-1,-1,-1,-1,0,Prisinformation\nPris per kvadratmeter\n22 667...,Kontakta mäklaren\nAmanda Gemmel\nSvensk Fasti...
2270,https://www.hemnet.se/salda/lagenhet-2rum-oste...,"Riddargatan 49A, 4tr","Östermalm, Stockholms kommun",Lägenhet,8000000,2022-07-20,123077,8450000,-450000,Andel i bostadsförening,...,"4 av 4, hiss finns ej",1893,1825,-1,-1,-1,-1,0,Prisinformation\nPris per kvadratmeter\n123 07...,Kontakta mäklaren\nOscar Bäverstam\nSMH Living
